# Practical 1
****
In this practical session, we begin to explore some of the concepts covered in the lectures and give you the opportunity to run some code.

Practical 1 focuses on building a logistic regression model and highlights some key steps and decisions that are needed when developing a logistic prediction model.

We will begin by loading our data set and perform some integrity checks. Some exploratory analysis will then be conducted to gain an understanding of the data. Once we are happy and familiar with the data we begin by showing you how to conduct univariate logistic regression models for categorical predictor variables. Following this we will explore univariate regression with a continuous variable and how to decide if the association between the dependent and independent variable is linear or non-linear. Finally, we will build a multivariable prediction model.


## The data

For todays practicals we will be using data from the Second Manifestations of ARTerial disease (SMART) study  
Here is a [link](https://link.springer.com/article/10.1023/A:1007621514757) to more information about the study.

| Variable | Description |
| :--- | :--- |
|TEVENT| Time of event (days) |
|EVENT| Cardiovascular event (0=no, 1=yes) |
|outcome| Cardiovascular event within 2 years of study entry (0=no, 1=yes) |
|SEX| 1 = male, 2 = female|
|AGE|Age (years)|
|DIABETES| Ever diabetes (0=no, 1=yes) |
|CEREBRAL| Ever cerebrovascular disease (0=no, 1=yes) |
|CARDIAC| Ever cardiovascular disease (0=no, 1=yes) |
|AAA| Ever abdominal aortic aneurysm (0=no, 1=yes) |
|PERIPH| Ever periferal aortic aneurysm (0=no, 1=yes) |
|STENOSIS| Ever periferal vascular disease (0=no, 1=yes) |
|SYSTBP| Systolic blood pressure (automatic, mm Hg)|
|DIASTBP| Diastolic blood pressure (automatic, mm Hg)|
|SYSTH| Systolic blood pressure (by hand, mm Hg)|
|DIASTH| Diastolic blood pressure (by hand, mm Hg)|
|LENGTHO| Height (m)|
|WEIGHTO| Weight (kg)|
|BMIO| Body mass index ($kg/m^2$)|
|CHOLO|Cholesterol level (mmol/L)|
|albumin| Albumin in urine (1=no, 2=low, 3=high)
|SMOKING| Smoking status (1=no, 2=Former, 3=current)
|packsyrs| Packetyears smoked|
|alcohol| Alcohol consumption (1=no, 2=Former, 3=current)|




The data was made freely available by Ewout Steyerberg who used the SMART data in his book 'Clinical prediction models: A Practical Approach to Development, Validation and Updating'. The data set is available from http://http://www.clinicalpredictionmodels.org/ but has been formated to make them easier for you to use. 

## Lets begin

First lets load the packages we will be using in the practical

In [ ]:
#install.packages("ggplot2")

In [ ]:
library(ggplot2)
library(plyr)
library(rms)
library(MASS)
library(reshape)
library(lmtest)

## Part 1 load and clean the data
Load the data and perform some data checks

The data is saved as an R data set (rds file), for information on how to load excel files please see the Intro to R HTML.

In [ ]:
smart <- readRDS(file= "SMARTs_P1.rds")
class(smart) # This tells us the class/data type of the object smart (hopefully returning data.frame)
dim(smart)
sapply(smart,class) # This tells us the class of each variable in the smart data, eg, categorical='factor' or continuous='numeric'

Albumin is not a factor variable so we need to change and label

In [ ]:
# Albumin
smart$albumin <- as.factor(smart$albumin)
levels(smart$albumin)
smart$albumin <- revalue(smart$albumin, c("1"="No", "2"="Low", "3"="High"))
levels(smart$albumin)

There are 2 variables for systolic blood pressure (SBP) because it can either done by hand or automatic. If we check, we can see there is a lot of missing data in both of the SBP variables.

In [ ]:
table(is.na(smart$SYSTBP)==FALSE)
table(is.na(smart$SYSTH)==FALSE)

However, if we look at how many individuals have SBP recorded by atleast one of the methods we find all but 3 have atleast one SBP measurement.

In [ ]:
table(is.na(smart$SYSTH)==FALSE | is.na(smart$SYSTBP)==FALSE)

 We can combine both of these variables to create a new SBP variable, such that, the variable will use the automatic SBP reading for each individual unless it is missing and then it will replace the missing indicator with the SBP recorded by hand.

In [ ]:
smart$SBP <- ifelse(is.na(smart$SYSTBP)==FALSE, smart$SYSTBP, smart$SYSTH)
table(is.na(smart$SBP))

Can you look at the diastolic blood pressure and the other variables?

## Part 2 Exploratory analysis
Now the data has been 'cleaned' we can begin to provide overviews of the data see the number of events/outcomes in our data.  

In [ ]:
# Overview of the data
attach(smart)
dim(smart)
summary(smart)
sapply(smart,class)
describe(smart)	

# Summarise outcome
table(outcome)
round(prop.table(table(outcome))*100,0)

We see that 212 (5%) individuals have the outcome of interest.  
It can be useful to also summarise each variable by the outcome

In [ ]:
# Summarise categorical variables by outcome
table(SEX,outcome) # Produces a table of counts
round(prop.table(table(SEX,outcome),2)*100,0) # Produces a table of percentages
round(prop.table(table(outcome,SEX),2)*100,0) # Produces a table of percentages

table(SMOKING,outcome) 
round(prop.table(table(SMOKING,outcome),2)*100,0)
round(prop.table(table(outcome,SMOKING),2)*100,0)

In [ ]:
# Look at distributions of continuous variables
summary(BMIO)
ggplot(smart,aes(x=BMIO))+geom_histogram()+facet_grid(~outcome)+theme_light()+ labs(title = "Histogram of age split by outcome")
ggplot(smart, aes(BMIO, fill = outcome)) + geom_histogram()
ggplot(smart, aes(BMIO, fill = as.factor(outcome))) + geom_density(alpha = 0.2)


### Question - Above we look into BMI, can you do the same for age?
What is the range of age values?  
Plot histogram of age  
Plot the density of age by outcome  

In [ ]:
# Insert R code here


## Part 3 Univariate logistic regression with categorical variables
Lets start by producing a univariate logistic model with gender as the independant variable.

In [ ]:
detach(smart)
# Univariable models for categorical variables

# Recode SEX so that the baseline is male #
smart$SEX2 = relevel(smart$SEX, ref="Male")
attach(smart)

# logistic regression model including sex as the only independent variable
sex_mod <- glm(outcome~SEX2,family="binomial")
summary(sex_mod)
beta <- round(exp(sex_mod$coef),3) # extract the model coefficients from the model
b <- round(exp(confint.default(sex_mod)),3) # Compute the confidence intervals for the model coefficients
d <- cbind(beta,b)
d

In [ ]:
summary(sex_mod)

In [ ]:
beta <- round(exp(sex_mod$coef),3) # extract the model coefficients from the model
beta

Can you interpret the output?

The estimate is the log odds.

**By modifying the code above, try fitting a logistic regression model to the SMART data with albumin as the predictor.**  
**Q. Interpret the output. Is albumin a significant predictor? What level of albumin is associated with the lowest risk?**

In [ ]:
# Insert R code here


## Part 4 Univariate logistic regression with continuous variables
Now lets try with a continuous variable (AGE).  

In [ ]:
# Assuming age is linear
age_mod <- glm(outcome~AGE,family="binomial")
summary(age_mod)
exp(confint.default((age_mod))) # if you want confidence intervals for the model coefficients
exp(0.03763)

If you wanted to calculate the models average predicted risk for a 40 year old, or an 80 year old, you can do this using the code below.

In [ ]:
lp40 <- predict(age_mod,data.frame(AGE=40))
risk40 <- exp(lp40)/(1+exp(lp40))

lp80 <- predict(age_mod,data.frame(AGE=80))
risk80 <- exp(lp80)/(1+exp(lp80))


matrix(c("risk40","risk80", risk40, risk80),ncol=2,byrow=F)

We see that on average, a 40 year old individual has a 2.5% risk of a cardiovascular event within 2 years.  
Alternatively, you can also use this code to obtain the risk for an individual who is 40 years old. The results are the same, it is just another way to calculate the risk.

In [ ]:
risk40a <- predict(age_mod,data.frame(AGE=40), type="response")*100 
# 'type="response"' gives the predicted probabilites and so multiply by 100 to get the risk as a percentage
risk40b <- 1/(1+exp(-lp40))*100 # alternative calculation
c(risk40a, risk40b)

## Part 5 Modelling a continuos variable using splines

Lets model age using splines. For information on splines, see Elements of stratistical learning book chapter 5.

Splines allow us to model non-linear relationships. They are piesewise polynomials, such that, polynomials are fit at different intervals along the variable of interest which is defined by pre-detewrmined points, knows as knots. 

Lets first create spline variables using the rcs command. We run the command rcs(AGE,n), where n refers to the number of knots. This will result in n-1 terms in the model output. For example, rcs(AGE,3) transforms age into a spline with 3 knots (automatically placed at the minimum, maximum and median age values) and 2 age terms.

Lets consiuder 3 different spline choices for age...

In [ ]:
age3_spline <- rcs(AGE,3)
age4_spline <- rcs(AGE,4)
age5_spline <- rcs(AGE,5)

Fit a logistic regression model using each set of spline variables & predict the linear predictor (LP)

In [ ]:
age3_mod <- glm(outcome~age3_spline,family="binomial")
summary(age3_mod)

In [ ]:
lp_age1 <- predict(age_mod)

age3_mod <- glm(outcome~age3_spline,family="binomial")
age3_mod
lp_age3 <- predict(age3_mod)

age4_mod <- glm(outcome~age4_spline,family="binomial")
age4_mod
lp_age4 <- predict(age4_mod)

age5_mod <- glm(outcome~age5_spline,family="binomial")
age5_mod
lp_age5 <- predict(age5_mod)

It is dificult to interpret output when using splines and easier to visualise.

Lets plot all LPs together for visual inspection (including the linear model)

In [ ]:
data_part6 <- data.frame(AGE,lp_age1,lp_age3,lp_age4,lp_age5)
data_part6_m <- melt(data_part6,id.vars='AGE')
plot_part6 <- ggplot(data_part6_m,aes(AGE,value,colour=variable))+geom_line()+scale_colour_manual(labels=c("linear","3 knots","4 knots","5 knots"),values=c("gray","green","red","blue"))+theme_bw()
plot_part6 + labs(x="Age (years)",y="Linear Predictor (log odds)",color="") + theme(legend.position=c(0.2,0.8))

From this graph which term would you use to model age? Do you think age is linear?  
We can check which is the best fitted model and most appropriate term to use for age by comparing AIC or BIC

Note, AIC is a measure of in sample prediction error that incorporates the goodness of fit and complexity of the model.

Both AIC and BIC are penalised-likelihood criteria, but BIC penalises model complexity more than AIC.

AIC = 2k - 2log(likelihood) 

In [ ]:
summary(age_mod)

In [ ]:
age_spline_check <- matrix(c(AIC(age_mod),
         BIC(age_mod),
         AIC(age3_mod),
         BIC(age3_mod),
         AIC(age4_mod),
         BIC(age4_mod),
         AIC(age5_mod),
         BIC(age5_mod)), ncol=2, byrow=TRUE)

colnames(age_spline_check) <- c("AIC", "BIC")
rownames(age_spline_check) <- c("age_mod","age3_mod", "age4_mod", "age5_mod")
age_spline_check

**Question - Using the above example, is there a non-linear relationship between BMI and the outcome? What term would you** **choose to model BMI? Do you think this makes sense biologically?**

In [ ]:
# Insert R code here

## Part 5 Building a multivariable model

Now we will move beyond univariate analysis and build a model using variable selection.

We will consider the following candidate predictors for inclusion in our model:  
SEX, AGE, SBP, alcohol, CHOLO, BMIO, DIABETES, CARDIAC, SMOKING, AAA  

Below we have subset the data to include only the data we need and removed any individuals with missing data.  
**Note: removing individuals with missing data has been done for simplicity and is not advised in practice. This can induce bias and one should always consider techniques such as multiple imputation if data are missing** 

In [ ]:
detach(smart)
smart <- subset(smart, select = c(outcome, SEX, AGE, SBP, alcohol, CHOLO, BMIO, DIABETES, CARDIAC, SMOKING, AAA))
smart <- na.omit(smart)
age3_spline <- rcs(smart$AGE,3)
attach(smart)

Next we can use forward or backward selection to develop a multivariable prediction model. We can use AIC to include/exclude predictors when building a model.

**Forward selection**, involves starting with no variables in the model, testing the addition of each variable using a chosen model fit criterion, adding the variable (if any) whose inclusion gives the most statistically significant improvement of the fit, and repeating this process until none improves the model to a statistically significant extent.

**Backward selection**, involves starting with all candidate variables, testing the deletion of each variable using a chosen model fit criterion, deleting the variable (if any) whose loss gives the most statistically insignificant deterioration of the model fit, and repeating this process until no further variables can be deleted without a statistically insignificant loss of fit.

Here we willconsider AIC as our criteria for variable selection and use a p-value of 0.1 (equivalent to a 2.706 change in AIC) so not to be too stringent. 


In [ ]:
k10 <- qchisq(0.10,1,lower.tail=FALSE)
k10

In [ ]:

#k10 <- qchisq(0.10,1,lower.tail=FALSE) # this give the change in AIC we consider to be significant in our stepwise selection

# Forward selection (by AIC)
empty_mod_2 <- glm(outcome~1,family="binomial")
forward_mod_2 <- stepAIC(empty_mod_2,k=k10,scope=list(upper=~SEX+age3_spline + SBP + alcohol + CHOLO + BMIO + DIABETES + CARDIAC + SMOKING + AAA,lower=~1),direction="forward",trace=TRUE)


In [ ]:
summary(forward_mod_2)

In [ ]:
exp(0.381)

**Can you interpret the output?**

Now lets try Backward selection.

**Can you also interpret this output?**

In [ ]:
# Backward selection (by AIC)
full_mod_2 <- glm(outcome~SEX+age3_spline + SBP + alcohol + CHOLO + BMIO + DIABETES + CARDIAC + SMOKING + AAA,family="binomial")
backward_mod_2 <- stepAIC(full_mod_2,k=k10,scope=list(upper=~SEX+age3_spline + SBP + alcohol + CHOLO + BMIO + DIABETES + CARDIAC + SMOKING + AAA,lower=~1),direction="backward",trace=TRUE)


In [ ]:
forward_mod_2
backward_mod_2

Here forward and backward selection choose the same model. However, this is not always what happens in practice.   
We can also force a variable to be included in the model.

In [ ]:
# Backward selection (by AIC) forcing SEX to be incuded in the model
k10 <- qchisq(0.1,1,lower.tail=FALSE)
backward_mod_2sex <- stepAIC(full_mod_2,k=k10,scope=list(upper=~SEX+age3_spline + SBP + alcohol + CHOLO + BMIO + DIABETES + CARDIAC + SMOKING + AAA,lower=~SEX),direction="backward",trace=TRUE)
backward_mod_2sex
summary(backward_mod_2sex)

In this practical we have built a model solely based on statistical results. However, in practice it is advised that both statistical and clinical relevance should be used to build a model. It is advised to build a model using clinical input and include variables that are clinically important even if the predictor does not appear statistically significant in the model